In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


# Keras for Neural Networks - Guided Example

Here we're going to work through a classic Machine Learning problem - digit recognition. This data is referred to as the MNIST dataset, which stands for Modified National Institute of Standards and Technology, and represents probably the most used dataset in the world for advanced machine learning techniques (though the iris dataset would be a close second). Here we're forgoing a more business focused dataset for a few reasons. Firstly, this dataset is the most written about dataset in these topics - you'll easily find other guides using pure TensorFlow or other tools like Theano to solve the same problem with the same class of models. Similarly, you can also easily find several different kinds of neural networks being used to solve this problem. This will be valuable as you try to expand your knowledge of different kinds of layers and combinations.

We'll be building our code off of the examples provided in the Keras documentation, and found in full on its [creator's github](https://github.com/fchollet/keras/tree/master/examples). 

Our goal here will be simple but multifaceted. Overall we are going to use the MNIST dataset and neural networks to classify handwritten numbers as the proper digits. This will be thought of as a multi-class classification problem, specifically with 10 classes (one for each possible digit).

However, we will use this to teach a few new kinds of neural network compositions, creating three different styles of network and discussing their relative advantages and disadvantages. Through this we will delve a little deeper into neural network theory.

But before we go too far, let's actually look at the data.

## MNIST DATA

In [2]:
# Import the dataset
from keras.datasets import mnist

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
type(y_train)

numpy.ndarray

In [6]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
import numpy as np
type(np.array([0,1,2]))

numpy.ndarray

When you look at this data you'll notice its organization structure is not images. We don't actually see any pictures of digits here. Instead, what we have is values of pixels, a simple way of converting images into numeric data on which we can train a model.

However, this still doesn't look like most of the data we've worked with previously. It's not a single table, but rather a different, higher dimensionality structure. It is often described as a set of clouds, each cloud representing an image. The cloud contains columns of values, representing the darkness of pixels. That's great, but not an easy or meaningful dataset on which to directly train a model. The darkness of the second pixel in the third column isn't likely linearly related to likelihood the cloud represents a certain digit. Instead, we need to find meaningful patterns within our clouds, creating models off of those patterns.

This is exactly what neural networks are good at. Multiple layers will allow us to transform this clouds full of values into meaningful vectors containing the information we need to be able to create a model, admittedly in an unlabeled and unsupervised fashion. Our output, however, will be labels for each of the clouds, giving us predictions as to what digit they are meant to represent.

Let's get started.

## Multi Layer Perceptron

Let's start with a kind of neural network we've seen before: a multi-layer perceptron. Recall from our previous neural networks sections that this is a set of perceptron models organized into layers, one layer feeding into the next.

To do this, we will first need to reshape our data into flat vectors for each digit. We'll also need to convert our outcome to a matrix of binary variables, rather than the digit.

In [14]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


Great. Now we can create our model. We'll do this using dense layers and dropouts. Dense layers are simply fully connected layers with a given number of perceptrons. Dropout drops a certain portion of our perceptrons in order to prevent overfitting. Our activation function, `relu` stands for Rectified Linear Unit, which is standard but can be read about more [here](https://en.wikipedia.org/wiki/Rectifier_(neural_networks).

In [6]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


Now we have a model. This we can use to accomplish our wildest dreams of data modeling, or at least predict some digits from pixel data. To do that we will use epochs, effectively iterations of the model, improving based on what it learned previously. Batch size is the number of samples to use in each step improving the model and will affect speed, but also slightly negatively impact accuracy (learning in bigger steps will affect what your model learns).

Note that we are going with 64 perceptron wide layers, this is relatively arbitrary, though units within the $2^x$ series will parallelize more efficiently. Also note that our number of parameters is the product of our input width plus one and our layer width. This reflects the number of weights we're creating in that layer.

In [16]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.0309 - accuracy: 0.9908 - val_loss: 0.1170 - val_accuracy: 0.9796
Epoch 2/10
60000/60000 [==============================] - 1s 22us/step - loss: 0.0348 - accuracy: 0.9897 - val_loss: 0.1167 - val_accuracy: 0.9791
Epoch 3/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.0323 - accuracy: 0.9903 - val_loss: 0.1270 - val_accuracy: 0.9789
Epoch 4/10
60000/60000 [==============================] - 1s 22us/step - loss: 0.0330 - accuracy: 0.9904 - val_loss: 0.1266 - val_accuracy: 0.9795
Epoch 5/10
60000/60000 [==============================] - 1s 22us/step - loss: 0.0322 - accuracy: 0.9903 - val_loss: 0.1257 - val_accuracy: 0.9783
Epoch 6/10
60000/60000 [==============================] - 1s 22us/step - loss: 0.0330 - accuracy: 0.9905 - val_loss: 0.1255 - val_accuracy: 0.9794
Epoch 7/10
60000/60000 [==============================] - 1s 22us/st

That did impressively well for such a simple neural network, with each epoch training in about 1 second on this machine and giving us an accuracy in the high 90's. But what else can we do? Let's let our model get much more complicated by introducting convolution.

## Convolutional Neural Networks

Before we go any further, do you recall that we've talked about how complex neural networks can get, and the degree of computational complexity that entails? Well, here we're going to finally truly experience that complexity, so be careful about rerunning this code. It will take some serious time (potentially on the order of hours) to run.

Now that that's out of the way, let's talk convolution. First, a simple definition. Convolution basically takes your data and creates overlapping subsegments testing for a given feature in a set of spaces and upon which it develops its model.

Let's extend that definition since it's incredibly dense.

First, you have to define a shape of your input data. This can theoretically be in any number of dimensions, though for our image example we will use 2d, since images are in two dimensions. This is also why you'll see 2D in some of our layer definitions (though more on that later). Our first chunk of code after loading the data does this reshaping (with a conditional on the data format).

Over that shaped data, we then create our tiles, also called __kernels__. These kernels are like little windows, that will look over subsets of the data of a given size. In the example below we create 3x3 kernels, which run overlapping over the whole 28x28 input looking for features. That is the convolutional layer, a way of searching for a subpattern over the whole of the image. We can chain multiple of these convolutional layers together, with the below example having two.

Next comes a pooling layer. This is a _downsampling_ technique, which effectively serves to reduce sample size and simplify later processes. For each value generated by our convolutional layers, it looks over the grid in _non_-overlapping segments and takes the maximum value of those outputs. It's not the feautres exact location then that matters, but its approximate or relative location. After pooling you will want to flatten the data back out, so that it can be put into dense layers as we did in MLP.

In [8]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.2534 - accuracy: 0.9210 - val_loss: 0.0554 - val_accuracy: 0.9816
Epoch 2/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0848 - accuracy: 0.9750 - val_loss: 0.0406 - val_accuracy: 0.9863
Epoch 3/10
60000/60000 [==============================] - 89s 1ms/step - loss: 0.0639 - accuracy: 0.9814 - val_loss: 0.0389 - val_accuracy: 0.9862
Epoch 4/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0541 - accuracy: 0.9840 - val_loss: 0.0326 - val_accuracy: 0.9884
Epoch 5/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0477 - accuracy: 0.9854 - val_loss: 0.0320 - val_accuracy: 0.9884
Epoch 6/10
60000/60000 [==============================] - 87s 1ms/step - loss: 0.0428 - accuracy: 0.9876 - val_loss: 0.0299 - val_accuracy: 0.

Now that is incredibly impressive accuracy. 99% is really exceptional, but it did take a long time to get there. Such are the costs of convolution.

There is one more classic construction of a neural network: Recurrent, which we'll give quick mention.

## Hierarchical Recurrrent Neural Networks

So far when we've talked about neural networks we've talked about them as feedforward: data flows in one direction until it reaches the end. Recurrent neural networks do not obey that directional logic, instead letting the data cycle through the network.

However, to do this we have to abandon the sequential model building we've done so far and things can get much more complicated. You have to use recurrent layers and often time distribution (which handles the extra dimension created through the LTSM layer, as a time dimension) to get these things running. You can find an example of a hierarchical recurrent network below (via the link [here](https://github.com/fchollet/keras/blob/master/examples/mnist_hierarchical_rnn.py)). When you get comfortable with networks as they exist in Keras for both convolution and MLP, start exploring recurrence. Note that this will take an even longer time than the previous ones should you choose to run it again.

In [9]:

# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 181s 3ms/step - loss: 0.9623 - accuracy: 0.6801 - val_loss: 0.3977 - val_accuracy: 0.8788
Epoch 2/3
60000/60000 [==============================] - 174s 3ms/step - loss: 0.3692 - accuracy: 0.8881 - val_loss: 0.2567 - val_accuracy: 0.9225
Epoch 3/3
60000/60000 [==============================] - 173s 3ms/step - loss: 0.2507 - accuracy: 0.9258 - val_loss: 0.1813 - val_accuracy: 0.9474
Test loss: 0.18129234880805015
Test accuracy: 0.9473999738693237


You should now be comfortable building some neural networks, but let's see if you can improve them!

# Drill: 99% MLP

We have the MLP above, which runs reasonably quickly. Copy that code down here and see if you can tune it to achieve 99% accuracy with a Multi-Layer Perceptron. Does it run faster than the recurrent or concolutional neural nets?

Note: Remember to reload the data to get it in the format that an MLP expects. HRNN takes data in a different format, but it is still called x_train and y_train in this notebook.

Our original MLP model had an out-of-sample accuracy of 0.977. First, the same model is estimated using 50 epochs instead of 10.

In [15]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 26us/step - loss: 0.4398 - accuracy: 0.8718 - val_loss: 0.1956 - val_accuracy: 0.9398
Epoch 2/50
60000/60000 [==============================] - 1s 24us/step - loss: 0.2038 - accuracy: 0.9387 - val_loss: 0.1389 - val_accuracy: 0.9559
Epoch 3/50
60000/60000 [==============================] - 2s 26us/step - loss: 0.1545 - accuracy: 0.9538 - val_loss: 0.1132 - val_accuracy: 0.9656
Epoch 4/50
60000/60000 [==============================] - 1s 22us/step - loss: 0.1279 - accuracy: 0.9613 - val_loss: 0.1048 - val_accuracy: 0.9694
Epoch 5/50
60000/60000 [==============================] - 1s 23us/step - loss: 0.1118 - accuracy: 0.9659 - val_loss: 0.0909 - val_accuracy: 0.9731
Epoch 6/50
60000/60000 [==============================] - 1s 25us/step - loss: 0.0998 - accuracy: 0.9686 - val_loss: 0.0981 - val_accuracy: 0.9717
Epoch 7/50
60000/60000 [==============================] - 1s 24us/st

Improved slightly. Next, I increase the dropout parameter in each level. I also returned to 10 epochs for ease of testing.

In [19]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.5580 - accuracy: 0.8313 - val_loss: 0.2224 - val_accuracy: 0.9310
Epoch 2/10
60000/60000 [==============================] - 1s 22us/step - loss: 0.2826 - accuracy: 0.9185 - val_loss: 0.1628 - val_accuracy: 0.9529
Epoch 3/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.2298 - accuracy: 0.9339 - val_loss: 0.1359 - val_accuracy: 0.9575
Epoch 4/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.2012 - accuracy: 0.9422 - val_loss: 0.1258 - val_accuracy: 0.9627
Epoch 5/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.1786 - accuracy: 0.9469 - val_loss: 0.1140 - val_accuracy: 0.9660
Epoch 6/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.1658 - accuracy: 0.9523 - val_loss: 0.1127 - val_accuracy: 0.9683
Epoch 7/10
60000/60000 [==============================] - 1s 24us/st

Trying with decreased dropout:

In [21]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.03))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.03))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.3866 - accuracy: 0.8888 - val_loss: 0.1936 - val_accuracy: 0.9415
Epoch 2/10
60000/60000 [==============================] - 1s 25us/step - loss: 0.1814 - accuracy: 0.9455 - val_loss: 0.1362 - val_accuracy: 0.9580
Epoch 3/10
60000/60000 [==============================] - 1s 23us/step - loss: 0.1343 - accuracy: 0.9594 - val_loss: 0.1131 - val_accuracy: 0.9659
Epoch 4/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.1086 - accuracy: 0.9669 - val_loss: 0.1072 - val_accuracy: 0.9687
Epoch 5/10
60000/60000 [==============================] - 1s 24us/step - loss: 0.0921 - accuracy: 0.9721 - val_loss: 0.1006 - val_accuracy: 0.9705
Epoch 6/10
60000/60000 [==============================] - 2s 25us/step - loss: 0.0817 - accuracy: 0.9747 - val_loss: 0.0995 - val_accuracy: 0.9709
Epoch 7/10
60000/60000 [==============================] - 2s 27us/st

Neither improved on our original accuracy rate. Try increasing the number of components in our first layer from 64 to 128

In [22]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(128, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 37us/step - loss: 0.3633 - accuracy: 0.8938 - val_loss: 0.1491 - val_accuracy: 0.9520
Epoch 2/10
60000/60000 [==============================] - 2s 30us/step - loss: 0.1605 - accuracy: 0.9512 - val_loss: 0.1132 - val_accuracy: 0.9638
Epoch 3/10
60000/60000 [==============================] - 2s 31us/step - loss: 0.1176 - accuracy: 0.9646 - val_loss: 0.0975 - val_accuracy: 0.9695
Epoch 4/10
60000/60000 [==============================] - 2s 30us/step - loss: 0.0948 - accuracy: 0.9714 - val_loss: 0.0881 - val_accuracy: 0.9743
Epoch 5/10
60000/60000 [==============================] - 2s 31us/step - loss: 0.0819 - accuracy: 0.9753 - val_loss: 0.0891 - val_accuracy: 0.9737
Epoch 6/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.0716 - accuracy: 0.9785 - val_loss: 0.0764 - val_accuracy: 0.9774
Epoch 7/10
60000/60000 [==============================] - 2s 28us/st

Increased further to 256:

In [24]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.3129 - accuracy: 0.9093 - val_loss: 0.1616 - val_accuracy: 0.9499
Epoch 2/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.1287 - accuracy: 0.9607 - val_loss: 0.0926 - val_accuracy: 0.9716
Epoch 3/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.0915 - accuracy: 0.9716 - val_loss: 0.0750 - val_accuracy: 0.9765
Epoch 4/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0704 - accuracy: 0.9785 - val_loss: 0.0745 - val_accuracy: 0.9775
Epoch 5/10
60000/60000 [==============================] - 2s 38us/step - loss: 0.0584 - accuracy: 0.9819 - val_loss: 0.0770 - val_accuracy: 0.9797
Epoch 6/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0483 - accuracy: 0.9848 - val_loss: 0.0749 - val_accuracy: 0.9798
Epoch 7/10
60000/60000 [==============================] - 2s 38us/st

Even a little bit better. Let's try increasing the size of the third layer from 64 to 128:

In [26]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 50us/step - loss: 0.2901 - accuracy: 0.9136 - val_loss: 0.1267 - val_accuracy: 0.9627
Epoch 2/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.1199 - accuracy: 0.9640 - val_loss: 0.0953 - val_accuracy: 0.9711
Epoch 3/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.0859 - accuracy: 0.9740 - val_loss: 0.0828 - val_accuracy: 0.9750
Epoch 4/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0641 - accuracy: 0.9803 - val_loss: 0.0747 - val_accuracy: 0.9794
Epoch 5/10
60000/60000 [==============================] - 3s 48us/step - loss: 0.0540 - accuracy: 0.9833 - val_loss: 0.0736 - val_accuracy: 0.9799
Epoch 6/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.0453 - accuracy: 0.9855 - val_loss: 0.0767 - val_accuracy: 0.9787
Epoch 7/10
60000/60000 [==============================] - 3s 53us/st

Out-of-sample accuracy decreased a little bit. Let's see if decreasing the size of the second layer has any effect:

In [27]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.3745 - accuracy: 0.8904 - val_loss: 0.1433 - val_accuracy: 0.9572
Epoch 2/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.1473 - accuracy: 0.9566 - val_loss: 0.1063 - val_accuracy: 0.9676
Epoch 3/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.1038 - accuracy: 0.9688 - val_loss: 0.0893 - val_accuracy: 0.9717
Epoch 4/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.0815 - accuracy: 0.9755 - val_loss: 0.0900 - val_accuracy: 0.9731
Epoch 5/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0677 - accuracy: 0.9791 - val_loss: 0.0738 - val_accuracy: 0.9782
Epoch 6/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.0589 - accuracy: 0.9821 - val_loss: 0.0665 - val_accuracy: 0.9787
Epoch 7/10
60000/60000 [==============================] - 2s 40us/st

Nope, still decreased. Looks like our second layer is pretty well tuned. Let's try widening our first layer even more.

In [29]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(512, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 81us/step - loss: 0.2736 - accuracy: 0.9183 - val_loss: 0.1253 - val_accuracy: 0.9608
Epoch 2/10
60000/60000 [==============================] - 5s 76us/step - loss: 0.1064 - accuracy: 0.9681 - val_loss: 0.0863 - val_accuracy: 0.9733
Epoch 3/10
60000/60000 [==============================] - 5s 75us/step - loss: 0.0753 - accuracy: 0.9768 - val_loss: 0.0762 - val_accuracy: 0.9768
Epoch 4/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.0578 - accuracy: 0.9821 - val_loss: 0.0772 - val_accuracy: 0.9779
Epoch 5/10
60000/60000 [==============================] - 4s 74us/step - loss: 0.0454 - accuracy: 0.9860 - val_loss: 0.0684 - val_accuracy: 0.9803
Epoch 6/10
60000/60000 [==============================] - 4s 75us/step - loss: 0.0369 - accuracy: 0.9886 - val_loss: 0.0728 - val_accuracy: 0.9817
Epoch 7/10
60000/60000 [==============================] - 5s 76us/st

Our model was starting to take a little longer to train, but this jump is when it was really felt (about 5s per epoch compared to less than 2 in our first model). Regardless, this set of parameters did worse, so I moved on assuming 256 was a fairly optimized parameter for the size of the first layer.

Next I played around with different loss functions. Most did not improve the model, but "binary crossentropy," did, and we finally achieved >99% accuracy.

In [47]:
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(256, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

# Compile the model to put it all together.
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 57us/step - loss: 0.0529 - accuracy: 0.9819 - val_loss: 0.0247 - val_accuracy: 0.9917
Epoch 2/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.0238 - accuracy: 0.9921 - val_loss: 0.0188 - val_accuracy: 0.9935
Epoch 3/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.0173 - accuracy: 0.9943 - val_loss: 0.0147 - val_accuracy: 0.9950
Epoch 4/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.0137 - accuracy: 0.9954 - val_loss: 0.0140 - val_accuracy: 0.9952
Epoch 5/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.0111 - accuracy: 0.9962 - val_loss: 0.0142 - val_accuracy: 0.9954
Epoch 6/10
60000/60000 [==============================] - 3s 45us/step - loss: 0.0097 - accuracy: 0.9969 - val_loss: 0.0123 - val_accuracy: 0.9960
Epoch 7/10
60000/60000 [==============================] - 3s 45us/st

For curiosities sake, I also tested to see if using the same loss function in a CNN produced better resultes.

In [48]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 105s 2ms/step - loss: 0.0676 - accuracy: 0.9765 - val_loss: 0.0160 - val_accuracy: 0.9946
Epoch 2/10
60000/60000 [==============================] - 94s 2ms/step - loss: 0.0240 - accuracy: 0.9921 - val_loss: 0.0099 - val_accuracy: 0.9965
Epoch 3/10
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0178 - accuracy: 0.9942 - val_loss: 0.0092 - val_accuracy: 0.9966
Epoch 4/10
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0141 - accuracy: 0.9954 - val_loss: 0.0073 - val_accuracy: 0.9973
Epoch 5/10
60000/60000 [==============================] - 95s 2ms/step - loss: 0.0123 - accuracy: 0.9959 - val_loss: 0.0070 - val_accuracy: 0.9973
Epoch 6/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.0063 - val_accuracy: 

The CNN with the categorical crossentropy loss function achieved 99.1% accuracy. With binary crossentropy, the CNN achieved 99.5% accuracy after just the first epoch, and after 10 epochs (and about 15 minutes), it achieved 99.8% accuracy.